In [1]:
from huggingface_hub import login

login()

In [1]:
model_path = "model\\gemma-3-ib-it"

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

datasets= load_dataset("json", data_files = "datasets.json") 
datasets = datasets["train"].train_test_split(test_size = 0.25)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, attn_implementation='eager')

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

model_name = model_path.split("/")[-1]

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    target_modules
     = ["q_proj", "v_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

def preprocess_function(examples):
    texts = []
    for conv in examples["messages"]:
        text = "\n".join([msg["role"] + ": " + msg["content"] for msg in conv])
        texts.append(text)
    model_inputs = tokenizer(texts, max_length=512, truncation=True, padding = "max_length")
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)


training_args = TrainingArguments(
    "trained_model",
    eval_strategy = "epoch",
    learning_rate=2e-4,
    save_strategy="no",   # epoch마다 저장
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs=5,
)

c:\Users\gus90\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\Users\gus90\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Map:   0%|          | 0/12991 [00:00<?, ? examples/s]

Map:   0%|          | 0/4331 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
)

trainer.train()

In [6]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3392.71


In [7]:
# trainer.push_to_hub()
trainer.save_model("model\\trained_model")
tokenizer.save_pretrained("model\\trained_model")

('model\\trained_model\\tokenizer_config.json',
 'model\\trained_model\\special_tokens_map.json',
 'model\\trained_model\\chat_template.jinja',
 'model\\trained_model\\tokenizer.model',
 'model\\trained_model\\added_tokens.json',
 'model\\trained_model\\tokenizer.json')

In [8]:
model_path = "model\\trained_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

text = input()

messages = [
    {"role" : "user", "content" : text},
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

저는 구글 딥마인드에서 훈련한 대규모 언어 모델인 Gemma입니다.<end_of_turn>
